# Advent of Code
## Day 7 - [No Space Left On Device](https://adventofcode.com/2022/day/7)

In [36]:
def parse_cmd_args(input):
    return [c.split(' ') for c in input.strip().split('\n')]

def parse_cmd(input):
    return [input[:2],parse_cmd_args(input[2:])]

def get_inputs(filename):
    with open(filename) as file:
        return [ parse_cmd(i) for i in file.read().strip().split('$ ') if i]

assert parse_cmd('cd /\n') == ['cd', [['/']]], 'change directory'
assert parse_cmd('ls\ndir a\n14848514 b.txt\n8504156 c.dat\ndir d\n') == ['ls', [['dir', 'a'], ['14848514', 'b.txt'], ['8504156', 'c.dat'],['dir','d']]], 'ls directory'

#print(get_inputs('../data/Day07-part1.txt'))